1. Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
2. 체인을 수동으로 구현해야 합니다.
3. 체인에 ConversationBufferMemory를 부여합니다.
4. 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
5. 체인에 다음 질문을 합니다:
    Q1. Aaronson 은 유죄인가요?
    Q2. 그가 테이블에 어떤 메시지를 썼나요?
    Q3. Julia 는 누구인가요?

문서 로드 > 분할 > 임베이딩 > 

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [3]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

In [4]:
cashe_dir = LocalFileStore(".cache/")

In [5]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 600,
    chunk_overlap = 100,
)

In [6]:
loader = UnstructuredFileLoader("document.txt")

In [7]:
docs = loader.load_and_split(text_splitter=splitter)

In [9]:
embeddings = OpenAIEmbeddings()

cashed_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cashe_dir)


In [10]:
vectorstore = FAISS.from_documents(docs, cashed_embeddings)

In [11]:
retriver = vectorstore.as_retriever()

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

In [13]:
chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


  Q1. Aaronson 은 유죄인가요?
    Q2. 그가 테이블에 어떤 메시지를 썼나요?
    Q3. Julia 는 누구인가요?

In [14]:
chain.invoke("Aaronson 은 유죄인가요?")

AIMessage(content='모르겠습니다.')

In [15]:
chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")

AIMessage(content='그는 테이블에 다음과 같은 메시지를 썼습니다:\n\nFREEDOM IS SLAVERY  \nTWO AND TWO MAKE FIVE  \nGOD IS POWER')

In [16]:
chain.invoke("Julia 는 누구인가요?")

AIMessage(content='문맥에서 Julia는 Winston의 사랑하는 사람으로 언급됩니다. 그녀는 Winston과 함께 자유롭게 있을 때보다 그를 더 사랑하게 된 인물입니다.')